In [7]:
%matplotlib widget

import matplotlib.pyplot as plt # For general plotting
from matplotlib import cm

from math import ceil, floor
import pandas as pd

import numpy as np
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
from scipy.stats import norm, multivariate_normal

np.set_printoptions(suppress=True)

plt.rc('font', size=22)          # controls default text sizes
plt.rc('axes', titlesize=18)     # fontsize of the axes title
plt.rc('axes', labelsize=18)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=14)    # fontsize of the tick labels
plt.rc('ytick', labelsize=14)    # fontsize of the tick labels
plt.rc('legend', fontsize=16)    # legend fontsize
plt.rc('figure', titlesize=22)   # fontsize of the figure title


# Set seed to generate reproducible "pseudo-randomness" (handles scipy's "randomness" too)
np.random.seed(7)

# Number of training input samples for experiments
N_train = [10, 100, 1000]
# Number of validation samples for experiments
N_valid = 10000

In [4]:
demographics_df = (
    pd.read_csv("demographics.csv")
    .assign(gender_ratio=lambda x: x.male / x.female)
    .drop(columns=["male", "female"])
)
features = [
    "hispanic_latino",
    "white",
    "black",
    "native_american",
    "asian",
    "nhpi",
    "other_race",
    "two_or_more",
    "income",
    "foreign_born",
    "gender_ratio",
    "bachelors",
    "median_age",
]
X = demographics_df[features]
y = demographics_df["margin"]

['hispanic_latino',
 'white',
 'black',
 'native_american',
 'asian',
 'nhpi',
 'other_race',
 'two_or_more',
 'income',
 'foreign_born',
 'gender_ratio',
 'bachelors',
 'median_age']

In [8]:
X_standardized = StandardScaler().fit_transform(X)
# The poly transform kind of breaks the notebook at the moment (refrain from running for now )
poly = PolynomialFeatures(len(features))
X_poly = poly.fit_transform(X_standardized)
X_train, X_test, y_train, y_test = train_test_split(X_standardized, y, random_state=0)

KeyboardInterrupt: 

In [5]:
# Define the logistic/sigmoid function
def sigmoid(z):
    return 1.0 / (1 + np.exp(-z))

# Define the prediction function y = 1 / (1 + np.exp(-X*w))
# X.dot(w) inputs to the sigmoid referred to as logits
def predict_prob(X, w):
    logits = X.dot(w)
    return sigmoid(logits)

In [ ]:
# NOTE: This implementation may encounter numerical stability issues...
# Read into the log-sum-exp trick OR use a class like: sklearn.linear_model import LogisticRegression
def log_reg_loss(w, X, y):
    # Size of batch
    B = X.shape[0]

    # Logistic regression model g(X * w)
    predictions = predict_prob(X, w)

    # NLL loss, 1/N sum [y*log(g(X*w)) + (1-y)*log(1-g(X*w))]
    error = predictions - y
    nll = -np.mean(y*np.log(predictions) + (1 - y)*np.log(1 - predictions))

    # Partial derivative for GD
    gradient = (1 / B) * X.T.dot(error)

    # Logistic regression loss, NLL (binary cross entropy is another interpretation)
    return nll, gradient


# Options for mini-batch gradient descent
opts = {}
opts['max_epoch'] = 100
opts['alpha'] = 0.05
opts['tolerance'] = 1e-3

opts['batch_size'] = 10

# Breaks the matrix X and vector y into batches
def batchify(X, y, batch_size, N):
    shuffled_indices = np.random.permutation(N) # Returns a permutation of indices up to N

    # Shuffle row-wise X (i.e. across training examples) and labels using same permuted order
    X = X[shuffled_indices]
    y = y[shuffled_indices]

    X_batch = []
    y_batch = []

    # Iterate over N in batch_size steps, last batch may be < batch_size
    for i in range(0, N, batch_size):
        nxt = min(i + batch_size, N + 1)
        X_batch.append(X[i:nxt, :])
        y_batch.append(y[i:nxt])

    return X_batch, y_batch


def gradient_descent(loss_func, theta0, X, y, N, *args, **kwargs):
    # Mini-batch GD. Stochastic GD if batch_size=1.

    # Break up data into batches and work out gradient for each batch
    # Move parameters theta in that direction, scaled by the step size.

    # Options for total sweeps over data (max_epochs),
    # and parameters, like learning rate and threshold.

    # Default options
    max_epoch = kwargs['max_epoch'] if 'max_epoch' in kwargs else 200
    alpha = kwargs['alpha'] if 'alpha' in kwargs else 0.1
    epsilon = kwargs['tolerance'] if 'tolerance' in kwargs else 1e-6

    batch_size = kwargs['batch_size'] if 'batch_size' in kwargs else 10

    # Turn the data into batches
    X_batch, y_batch = batchify(X, y, batch_size, N)
    num_batches = len(y_batch)
    print("%d batches of size %d\n" % (num_batches, batch_size))

    theta = theta0
    m_t = np.zeros(theta.shape)

    trace = {}
    trace['loss'] = []
    trace['theta'] = []

    # Main loop:
    for epoch in range(1, max_epoch + 1):
        # print("epoch %d\n" % epoch)

        loss_epoch = 0
        for b in range(num_batches):
            X_b = X_batch[b]
            y_b = y_batch[b]
            # print("epoch %d batch %d\n" % (epoch, b))

            # Compute NLL loss and gradient of NLL function
            loss, gradient = loss_func(theta, X_b, y_b, *args)
            loss_epoch += loss

            # Steepest descent update
            theta = theta - alpha * gradient

            # Terminating Condition is based on how close we are to minimum (gradient = 0)
            if np.linalg.norm(gradient) < epsilon:
                print("Gradient Descent has converged after {} epochs".format(epoch))
                break

        # Storing the history of the parameters and loss values per epoch
        trace['loss'].append(np.mean(loss_epoch))
        trace['theta'].append(theta)

        # Also break epochs loop
        if np.linalg.norm(gradient) < epsilon:
            break

    return theta, trace